In [1]:
%cd C:/Users/realc/OneDrive/Documents/GSOC

from utils.post_training import *
from utils.statistics import *
from utils.plotting import *
from utils.encodings import *
from utils.statistics import calculate_statistics, calculate_fid, ssim
from utils.haar_noising_script import apply_global_haar_scrambling
from utils.quantum_diffusion import *

import numpy as np
import h5py
import torch
import torch.nn as nn
import torch.optim as optim

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import scipy.linalg

import pennylane as qml

c:\Users\realc\OneDrive\Documents\GSOC\.venv\Lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


C:\Users\realc\OneDrive\Documents\GSOC


In [3]:
QG_channel = 1
filename = f"data/QG{QG_channel}_64x64_1k"
num_samples = 10
seed = 42
n_qubits = 12
results_file = f"data/QG{QG_channel}_results.csv"

data_X = np.array(h5py.File(filename, "r")['X'])
print("Data shape:", data_X.shape)

Data shape: (1000, 64, 64)


In [ ]:
# Do not run this unless needing to reencode and rescramble everything
encoding_methods = {
    "angle": angle_encoding,
    "amplitude": amplitude_encoding,
    "sinusoidal": sinusoidal_encoding,
    "iqp": iqp_encoding,
}

def encode_dataset(encoding_fn, data, num_samples, n_qubits):
    """Encode all samples using a given encoding function."""
    dev = qml.device("default.qubit", wires=n_qubits)

    @qml.qnode(dev)
    def circuit(image):
        encoding_fn(image, wires=range(n_qubits))
        return qml.state()

    encoded = [circuit(data[i]) for i in range(num_samples)]
    return np.array(encoded)

# Loop through all encodings
for name, enc_fn in encoding_methods.items():
    print(f"\n=== Running with {name} encoding ===")
    
    if enc_fn == "angle":
        n_qubits = 4096
    
    encoded_data = encode_dataset(enc_fn, data_X, num_samples, n_qubits)
    # encoded_data_torch = torch.tensor(encoded_data, dtype=torch.float32)
    
    scrambled_states, used_unitaries = apply_global_haar_scrambling(np.array(encoded_data), seed=42, approx_threshold_qubits=200)
    scrambled_torch = torch.tensor(scrambled_states, dtype=torch.float32)

    torch.save(torch.tensor(encoded_data, dtype=torch.float32), f"data/QG{QG_channel}_64x64_{num_samples}_{name}_encoded.pt")
    torch.save(scrambled_torch, f"data/QG{QG_channel}_64x64_{num_samples}_{name}_scrambled.pt")


Data shape: (1000, 64, 64)

=== Running with angle encoding ===

=== Running with amplitude encoding ===

=== Running with sinusoidal encoding ===

=== Running with iqp encoding ===


In [ ]:
# Define encodings to test
encoding_methods = {
    "angle": angle_encoding,
    "amplitude": amplitude_encoding,
    "sinusoidal": sinusoidal_encoding,
    "iqp": iqp_encoding,
}

def encode_dataset(encoding_fn, data, num_samples, n_qubits):
    """Encode all samples using a given encoding function."""
    dev = qml.device("default.qubit", wires=n_qubits)

    @qml.qnode(dev)
    def circuit(image):
        encoding_fn(image, wires=range(n_qubits))
        return qml.state()

    encoded = [circuit(data[i]) for i in range(num_samples)]
    return np.array(encoded)

# Loop through all encodings
for name, enc_fn in encoding_methods.items():
    print(f"\n=== Running with {name} encoding ===")
    
    if enc_fn == "angle":
        n_qubits = 4096
    
    encoded_data = encode_dataset(enc_fn, data_X, num_samples, n_qubits)
    # encoded_data_torch = torch.tensor(encoded_data, dtype=torch.float32)
    
    scrambled_states, used_unitaries = apply_global_haar_scrambling(np.array(encoded_data), seed=42, approx_threshold_qubits=200)
    scrambled_torch = torch.tensor(scrambled_states, dtype=torch.float32)

    torch.save(torch.tensor(encoded_data, dtype=torch.float32), f"data/QG{QG_channel}_64x64_{num_samples}_{name}_encoded.pt")
    torch.save(scrambled_torch, f"data/QG{QG_channel}_64x64_{num_samples}_{name}_scrambled.pt")


In [ ]:
encodings = ["angle", "amplitude", "sinusoidal", "iqp"]
results = []

for enc in encodings:
    print(f"\n=== {enc} encoding ===")
    encoded = torch.load(f"data/QG{QG_channel}_64x64_{num_samples}_{enc}_encoded.pt")
    scrambled = torch.load(f"data/QG{QG_channel}_64x64_{num_samples}_{enc}_scrambled.pt")
    print("Loaded encoded and scrambled data.")

    train_enc, val_enc, train_scr, val_scr = train_test_split(
        encoded, scrambled, test_size=0.2, random_state=42, shuffle=True
    )

    model, reconstructed = train_model(train_enc, val_enc, train_scr, val_scr)

    # Metrics
    mu_real, sigma_real = calculate_statistics(data_X[:len(reconstructed)])
    mu_fake, sigma_fake = calculate_statistics(reconstructed)
    fid = calculate_fid(mu_real, sigma_real, mu_fake, sigma_fake)
    mean_ssim = calculate_ssim(data_X, reconstructed)

    results.append({"encoding": enc, "FID": fid, "SSIM": mean_ssim})



=== angle encoding ===
Loaded encoded and scrambled data.


C:\Users\realc\AppData\Local\Temp\ipykernel_10016\3037138951.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  encoded = torch.load(f"data/QG{QG_channel}_64x64_{num_sample

Epoch [1/50] - Loss: 0.0046, Val Loss: 0.0025
